# pytorch basics tutorial
see: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
import torch
from torch import nn

# define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
  )
)


In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
'''
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")
'''

'\nloss_fn = nn.CrossEntropyLoss()\noptimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)\n\nepochs = 10\nfor t in range(epochs):\n    print(f"Epoch {t+1}\n-------------------------------")\n    train_loop(train_dataloader, model, loss_fn, optimizer)\n    test_loop(test_dataloader, model, loss_fn)\nprint("Done!")\n'

# import data

In [7]:
import pandas as pd

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    df = pd.read_csv("nnbasic-btc-data.csv", index_col=0).reset_index(drop=True)
else:
    df = pd.read_csv("nnbasic-btc-data.csv", index_col=0).reset_index(drop=True)


print(df.shape)
df.head(3)

(11713, 68)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3,log_open_s3,log_high_s3,log_low_s3,log_close_s3,...,log_high,log_low,log_close,log_vol,norm_open,norm_high,norm_low,norm_close,norm_vol,norm_log_vol
0,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675,10.207403,10.207588,10.206573,10.207175,...,10.206698,10.205620,10.206698,5.385151,0.378108,0.313275,0.383429,0.382019,0.003001,0.276083
1,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680,10.207175,10.207175,10.204950,10.205331,...,10.207802,10.206311,10.207289,5.797309,0.382019,0.317588,0.385941,0.384170,0.004729,0.328539
2,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080,10.205331,10.206455,10.205110,10.205623,...,10.209464,10.207289,10.208852,6.887200,0.384170,0.324095,0.389502,0.389869,0.014827,0.467251


In [8]:
# do not use the time columns 

df_train = df.filter(items = ['norm_open_s3', 'norm_high_s3', 'norm_low_s3', 'norm_close_s3', 'norm_log_vol_s3',
       'norm_open_s2', 'norm_high_s2', 'norm_low_s2', 'norm_close_s2', 'norm_log_vol_s2',
       'norm_open_s1', 'norm_high_s1', 'norm_low_s1', 'norm_close_s1', 'norm_log_vol_s1', 
       'norm_open', 'norm_high', 'norm_low', 'norm_close', 'norm_log_vol']).dropna()
print(df_train.shape)
df_train.head(3)

(11713, 20)


,norm_open_s3,norm_high_s3,norm_low_s3,norm_close_s3,norm_log_vol_s3,norm_open_s2,norm_high_s2,norm_low_s2,norm_close_s2,norm_log_vol_s2,norm_open_s1,norm_high_s1,norm_low_s1,norm_close_s1,norm_log_vol_s1,norm_open,norm_high,norm_low,norm_close,norm_log_vol
0,0.384586,0.316752,0.386895,0.383753,0.348938,0.383753,0.315136,0.380997,0.377046,0.355982,0.377046,0.312322,0.381575,0.378108,0.305599,0.378108,0.313275,0.383429,0.382019,0.276083
1,0.383753,0.315136,0.380997,0.377046,0.355982,0.377046,0.312322,0.381575,0.378108,0.305599,0.378108,0.313275,0.383429,0.382019,0.276083,0.382019,0.317588,0.385941,0.384170,0.328539
2,0.377046,0.312322,0.381575,0.378108,0.305599,0.378108,0.313275,0.383429,0.382019,0.276083,0.382019,0.317588,0.385941,0.384170,0.328539,0.384170,0.324095,0.389502,0.389869,0.467251


# pytorch beginner tutorial by jeremy howard

https://pytorch.org/tutorials/beginner/nn_tutorial.html

### Sidebar: bridging the gap between NNs for images and trading data
**Imagine that every da is like a number** an that it is a picture with uniform height (maximum distance of open price in the data set) and width (=time). If this is the case then there may be are more straight forward links to prices and predicting days based on the days before yields better results? Another option is to categorize the price movements. 

In [9]:
y = df_train.pop("norm_high")
x = df_train

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [11]:
from torch import tensor
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train.values, y_train.values, x_test.values, y_test.values) )

In [12]:
n, c = x_train.shape
print(f"x_train.shape: {x_train.shape} , y_train.shape: {y_train.shape}")
print(y_train.min(), y_train.max(), "because it was put through a log function and minMax scaler")

x_train.shape: torch.Size([8784, 19]) , y_train.shape: torch.Size([8784])
tensor(0., dtype=torch.float64) tensor(0.9903, dtype=torch.float64) because it was put through a log function
